Trains a RNN on medical diagnosis of diseases dataset data is obtained from various online sources  
Memory network needs to predict the disease using many symptoms listed as natural language sentences

# Imports

In [1]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('stopwords')

In [2]:
# !python -m spacy download en_core_web_sm

In [1]:
import re
import csv
import pdb
import h5py
import json
import string
import inflect # convert number into words
import pickle
import tarfile
import itertools
from functools import reduce

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
import transformers
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_excel('../datasets/updated details 2.xlsx', index_col=0)
df.head(1)

,disease,synonym_disease,synonym_disease2,link,remedies,overview,treatment
0,acid reflux,gastroesophageal reflux disease,GERD,https://www.mayoclinic.org/diseases-conditions...,Maintain a healthy weight. Excess pounds put p...,Overview.\n\nGastroesophageal reflux disease (...,Diagnosis.\n\nYour health care provider might ...


In [5]:
diseases = df.disease.values
diseases_overview = df.overview.values

diseases.shape, diseases_overview.shape

((47,), (47,))

In [6]:
diseases_overview[1]

"Overview.\n\nAcne is a skin condition that occurs when your hair follicles become plugged with oil and dead skin cells. It causes whiteheads, blackheads or pimples. Acne is most common among teenagers, though it affects people of all ages.\n\nEffective acne treatments are available, but acne can be persistent. The pimples and bumps heal slowly, and when one begins to go away, others seem to crop up.\n\nDepending on its severity, acne can cause emotional distress and scar the skin. The earlier you start treatment, the lower your risk of such problems.\n\nSymptoms.\n\nAcne signs vary depending on the severity of your condition:\n\nWhiteheads (closed plugged pores)\nBlackheads (open plugged pores)\nSmall red, tender bumps (papules)\nPimples (pustules), which are papules with pus at their tips\nLarge, solid, painful lumps under the skin (nodules)\nPainful, pus-filled lumps under the skin (cystic lesions).\n\nWhiteheads (closed plugged pores).\n\nBlackheads (open plugged pores).\n\nSmall r

In [68]:
sentence_embeddings = {}

for i in range(47):
    try:
        sentence_embeddings[diseases[i]] = model.encode(preprocess_pipe(diseases_overview[i]))
    except:
        pass

In [80]:
query = "i have acne"
query_embedding = model.encode(preprocess_pipe(query))

In [81]:
scores = {}
for key, val in sentence_embeddings.items():
    scores[key] = util.cos_sim(query_embedding, val)

In [82]:
dict(sorted(scores.items(),key=lambda item:item[1]))

{'bee sting': tensor([[0.1231]]),
 'stings': tensor([[0.1231]]),
 'obesity': tensor([[0.1290]]),
 'common cold': tensor([[0.1337]]),
 'head lice': tensor([[0.1414]]),
 'premenstrual syndrome': tensor([[0.1426]]),
 'body lice': tensor([[0.1585]]),
 'cavities': tensor([[0.1651]]),
 'hiccups': tensor([[0.1782]]),
 'bad breath': tensor([[0.1870]]),
 'allergies': tensor([[0.1901]]),
 'snoring': tensor([[0.1913]]),
 'panic attacks and panic disorder': tensor([[0.1919]]),
 'mosquito bites': tensor([[0.1920]]),
 'dengue fever': tensor([[0.1947]]),
 'calluses and corns': tensor([[0.2031]]),
 'corns and calluses': tensor([[0.2031]]),
 'body odor and sweating': tensor([[0.2086]]),
 'menstrual cramps': tensor([[0.2160]]),
 'polycystic ovary syndrome': tensor([[0.2232]]),
 'muscle cramp': tensor([[0.2238]]),
 'indigestion': tensor([[0.2269]]),
 'constipation': tensor([[0.2320]]),
 'acid reflux': tensor([[0.2399]]),
 'alopecia': tensor([[0.2409]]),
 'baldness': tensor([[0.2409]]),
 'myopia': tensor(

# 1 preprocess text

1. Converting to lowercase
2. Converting digits to words
3. Remove punctuation an whitespace
4. Removing default stopwords
5. Lemmatization

## 1.1 Converting to lowercase
Happy > happy

In [58]:
def text_lowercase(text):
    return text.lower()

## 1.2 Converting digits to words
3 > three

In [59]:
def convert_number(text):
    p = inflect.engine()
    temp_str = text.split()

    new_string = []

    for word in temp_str:
        if word.isdigit():
            temp = p.number_to_words(word)
            new_string.append(temp)

        else:
            new_string.append(word)

    temp_str = ' '.join(new_string)
    return temp_str

## 1.3 Remove punctuation and whitespace
itching   ! > itching

In [60]:
def remove_punctuation(text):
    text = text.replace('_', ' ')
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [61]:
def remove_whitespace(text):
    return  " ".join(text.split())

## 1.4 Removing default stopwords

In [62]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

## 1.5 Lemmatization
itching > itch

In [63]:
def lemmatize_word(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)

    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return " ".join(lemmas)

In [67]:
def preprocess_pipe(text):
    """
    Combining all preprocessing steps.
    """
    text = text_lowercase(text)
    text = convert_number(text)
    text = remove_punctuation(text)
    text = remove_whitespace(text)
    text = remove_stopwords(text)
    text = lemmatize_word(text)

    return text

## 1.6 Preprocessing all symptoms

In [ ]:
symp = []
disease = []
for i in range(len(df_tr)):
    symp.append(df_tr.columns[df_tr.iloc[i] == 1].to_list())
    disease.append(df_tr.iloc[i, -1])

In [ ]:
# preprocessing all symptoms
all_symp = []
for sis in symp:
    temp = []
    for s in sis:
        temp.append(preprocess_sym(s))
    all_symp.append(temp)

In [ ]:
all_symp

In [ ]:
# associates each preprocessed symp with the disease
col_dict = dict(zip(disease, all_symp))

In [ ]:
col_dict

# 3 - syntactic similarity

In [ ]:
def pattern_set(str1, str2):
    list1 = str1.split(' ')
    list2 = str2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
# syn similarity with the corpus
def syntactic_similarity(symp_t, corpus):
    most_sim = []
    poss_sym = []

    for symp in corpus:
        d = pattern_set(symp_t, symp)
        most_sim.append(d)

    order = np.argsort(most_sim)[::-1].tolist()

    for i in order:
        if DoesExist(symp_t):
            return 1, [corpus[i]]

        if corpus[i] not in poss_sym and most_sim[i] != 0:
            poss_sym.append(corpus[i])

    if len(poss_sym):
        return 1, poss_sym


    else:
        return 0, None

In [ ]:
# Returns all the subsets of this set. This is a generator.
def powerset(seq):
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [ ]:
# Sort list based on length
def sort(a):
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if len(a[j])>len(a[i]):
                a[i],a[j]=a[j],a[i]
    a.pop()
    return a

In [ ]:
# find all permutations of a list
def permutations(s):
    permutations = list(itertools.permutations(s))
    return([' '.join(permutation) for permutation in permutations])

In [ ]:
def DoesExist(txt):
    txt=txt.split(' ')
    combinations = [x for x in powerset(txt)]
    sort(combinations)

    for comb in combinations :
        # print(permutations(comb))
        for sym in permutations(comb):
            if sym in all_symp_pr:
                # print(sym)
                return sym
    return False

In [ ]:
DoesExist('worried')

In [ ]:
preprocess_sym('really worried')

In [ ]:
syntactic_similarity(preprocess_sym('nervous'), all_symp_pr)

In [ ]:
def check_pattern(inp,dis_list):
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        if regexp.search(item):
            pred_list.append(item)
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,None

In [ ]:
check_pattern('nail', all_symp_pr)

# 4 - semantic similarity

In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
def sentence_similarity(sentence1, sentence2):
    # encode list of sentences to get their embeddings
    embedding1 = model.encode(sentences1, convert_to_tensor=True)
    embedding2 = model.encode(sentences2, convert_to_tensor=True)

    # compute similarity scores of two embeddings
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    return cosine_scores

In [ ]:
sentence_similarity('tensed', 'nervous')

In [ ]:
def semantic_similarity(symp_t, corpus):
    sims = {}
    for symp in corpus:
        d = sentence_similarity(symp_t, symp)
        sims[symp] = d

    return sims

In [ ]:
all_symp_pr.sort()
semantic_similarity('tensed', all_symp_pr)

In [ ]:
all_symp_pr.sort()

In [ ]:
WSD('season', 'apply spices to the chicken to season it').definition()

In [ ]:
all_symp_pr

In [ ]:
def suggest_syn(sym):
    symp = []
    synonyms = wordnet.synsets(sym)
    lemmas=[word.lemma_names() for word in synonyms]
    lemmas = list(set(chain(*lemmas)))
    for e in lemmas:
        res,sym1=semantic_similarity(e,all_symp_pr)
        if res != 0:
            symp.append(sym1)
    return list(set(symp))

In [ ]:
suggest_syn('worried')

In [ ]:
# recoit client_symptoms et renvoit un dataframe avec 1 pour les symptoms associees
def OHV(cl_sym,all_sym):
    l=np.zeros([1,len(all_sym)])
    for sym in cl_sym:
        l[0,all_sym.index(sym)]=1
    return pd.DataFrame(l, columns =all_symp)

In [ ]:
def contains(small, big):
    a=True
    for i in small:
        if i not in big:
            a=False
    return a

In [ ]:
def possible_diseases(l):
    poss_dis=[]
    for dis in set(disease):
        if contains(l,symVONdisease(df_tr,dis)):
            poss_dis.append(dis)
    return poss_dis

In [ ]:
set(disease)

In [ ]:
#recoit une maladie renvoit tous les sympts
def symVONdisease(df,disease):
    ddf=df[df.prognosis==disease]
    m2 = (ddf == 1).any()
    return m2.index[m2].tolist()
    

In [ ]:
symVONdisease(df_tr,'Jaundice')

# 4 - Utility functions

In [ ]:
def check_repeated(name,repeated_list):
    name = name.lower().strip()
    return name if not (name in repeated_list) else repeated_list[name]

In [ ]:
def process_title(word):
    return re.sub(r'\W+', ' ', word).strip().lower()

In [ ]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip().lower() for x in re.split('(\W+)?', sent) if x.strip()]

In [ ]:
def parse_stories(lines, only_supporting=False,repeated_list=None):
    '''Parse stories
    If only_supporting is true, only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        spl = line.split(' ', 1)
        if len(spl) > 1:
            nid, line = spl
        else:
            continue
        try:
            nid = int(nid)
        except ValueError:
            pdb.set_trace()
        if nid == 0:
            story = []
        if '\t' in line:
            supporting, a = line.split('\t')
            a = map(process_title,a.split(','))
            options = [] if len(a) == 1 else list(set(a[1:]))
            a = a[0]    
            substory = None
            # Provide all the substories
            if supporting:
                story.append([tokenize(supporting) + [u'.']])
            substory = [x for x in story if x]
            # TODO: I should have done the lower in previous processing steps
            if not substory:
                continue
            data.append((substory, a.lower(), map(lambda x:x.lower(),options)))
        else:
            sent = tokenize(line)
            story.append([sent + [u'.']])
    return data

In [ ]:
def get_stories(f, only_supporting=False, max_length=None,repeated_list=None, min_length=None):
    '''Given a file name, read the file, retrieve the stories, and then convert the sentences into a single story.
    If max_length is supplied, any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting,repeated_list=repeated_list)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [[flatten(reversed(story)), answer, options] for story,answer,options in data if not max_length or len(flatten(story)) < max_length]
    # At least two facts
    print(len(data))
    if min_length: 
        data = filter(lambda x: len(x[0]) > min_length, data)
    print(len(data))
    return data

In [ ]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        y = np.zeros(len(word_idx))  # let's not forget that index 0 is reserved
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [ ]:
def get_spacy_vectors(data, answer_dict, story_maxlen, model):
    X = []
    Y = []
    for story,answer in data:
        story = story[:story_maxlen] if len(story) > story_maxlen else story
        x = [model(unicode(w)).vector for w in story]
        X.append(x)
        if not answer_dict is None:
            y = np.zeros(len(answer_dict))
            y[answer_dict[answer]] = 1
            Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen,dtype='float32'),
            np.array(Y))

In [ ]:
def get_word_vectors(data, answer_dict, story_maxlen, model):
    X = []
    Y = []
    for story,answer in data:
        story = story[:story_maxlen] if len(story) > story_maxlen else story
        x = [model.word_vectors[model.dictionary[w]] for w in story if 
             not model.dictionary.get(w) is None]
        X.append(x)
        if not answer_dict is None:
            y = np.zeros(len(answer_dict))
            y[answer_dict[answer]] = 1
            Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen,dtype='float32'),
            np.array(Y))

In [ ]:
def create_vectors_dataset(input_files, vector_files, max_len=500):
    print('Creating word vectors file')

    training_set_file, test_set_file = input_files
    train_word_file, test_word_file = vector_files
    
    train_stories = pickle.load(open(training_set_file,'r'))
    test_stories = pickle.load(open(test_set_file,'r'))

    train_stories = [(reduce(lambda x,y: x + y, map(list,fact)),q) for fact,q in train_stories]
    test_stories = [(reduce(lambda x,y: x + y, map(list,fact)),q) for fact,q in test_stories]

    vocab = sorted(reduce(lambda x, y: x | y, (set(story + [answer]) for story, answer in train_stories + test_stories)))

    # Reserve 0 for masking via pad_sequences
    vocab_size = len(vocab) + 1
    story_maxlen = max(map(len, (x for x, _ in train_stories + test_stories)))


    print('-')
    print('Vocab size:', vocab_size, 'unique words')
    print('Story max length:', story_maxlen, 'words')
    print('Number of training stories:', len(train_stories))
    print('Number of test stories:', len(test_stories))
    print('-')
    print('Here\'s what a "story" tuple looks like (input, query, answer):')
    print(train_stories[0])
    print('-')
    print('Vectorizing the word sequences...')

    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))

    answer_vocab = sorted(reduce(lambda x, y: x | y, (set([answer]) for _, answer in train_stories + test_stories)))
    # Reserve 0 for masking via pad_sequences
    answer_dict = dict((word, i) for i, word in enumerate(answer_vocab))
    print('Answers dict len: {0}'.format(len(answer_dict)))

    # I need to check also if this exist
    word_vectors_dir = 'word_vectors/glove.42B.300d.txt'
    word_vectors_model = Glove.load_stanford(word_vectors_dir)

    inputs_train, answers_train = get_word_vectors(train_stories, answer_dict, 
                                                   max_len, word_vectors_model)
    inputs_test, answers_test = get_word_vectors(test_stories, answer_dict, max_len,
                                                 word_vectors_model)

    with h5py.File(train_word_file,'w') as train_f:
        _ = train_f.create_dataset('inputs',data=inputs_train)
        _ = train_f.create_dataset('answers',data=answers_train)
    with h5py.File(test_word_file,'w') as test_f:
        _ = test_f.create_dataset('inputs',data=inputs_test)
        _ = test_f.create_dataset('answers',data=answers_test)
        
    return (inputs_train, answers_train),(inputs_test, answers_test)

In [ ]:
def save_vectors_dict(input_files):
    # I need to check also if this exist
    filename = 'word_vectors/glove.42B.300d.txt'
    word_vectors_dict = 'word_vectors/glove_dict.hdf5'
    dct = {}
    vectors = array.array('d')

    # Read in the data.
    with io.open(filename, 'r', encoding='utf-8') as savefile:
        for i, line in enumerate(savefile):
            tokens = line.split(' ')

            word = tokens[0]
            entries = tokens[1:]

            dct[word] = i
            vectors.extend(float(x) for x in entries)
            
    print('Saving to hf5 file')
    with h5py.File(word_vectors_dict,'w') as vector_f:
        _ = vector_f.create_dataset('vectors',data=dct)

# 4 - Prediction Model

In [ ]:
RNN = recurrent.LSTM
NUM_HIDDEN_UNITS = 128
BATCH_SIZE = 32
EPOCHS = 10
DROPOUT_FACTOR = 0.2

In [ ]:
max_len = 500
word_vec_dim = 300
vocab_size = 2350

In [ ]:
training_set_file = 'data/training_set.dat'
test_set_file = 'data/test_set.dat'

train_stories = pickle.load(open(training_set_file,'r'))
test_stories = pickle.load(open(test_set_file,'r'))

train_stories = [(reduce(lambda x,y: x + y, map(list,fact)),q) for fact,q in train_stories]
test_stories = [(reduce(lambda x,y: x + y, map(list,fact)),q) for fact,q in test_stories]

In [ ]:
answer_vocab = sorted(reduce(lambda x, y: x | y, (set([answer]) for _, answer in train_stories + test_stories)))

# Reserve 0 for masking via pad_sequences
answer_dict = dict((word, i) for i, word in enumerate(answer_vocab))
print('Answers dict len: {0}'.format(len(answer_dict)))

In [ ]:
# I need to check also if this exist
# word_vectors_dir = 'word_vectors/glove.42B.300d.txt'
# word_vectors_model = Glove.load_stanford(word_vectors_dir)
nlp = English()

## Model

In [ ]:
model = Sequential()

# GRU
model.add(GRU(
    output_dim=NUM_HIDDEN_UNITS,
    activation='tanh',
    return_sequences=True,
    input_shape=(max_len, word_vec_dim)
))

# Dropout
model.add(Dropout(DROPOUT_FACTOR))

# GRU
model.add(GRU(
    NUM_HIDDEN_UNITS,
    return_sequences=False
))

# Dense
model.add(Dense(
    vocab_size,
    init='uniform',
    activation='softmax'
))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
base_dir = '.'

NUM_DATA_TRAIN = len(train_stories)
NUM_DATA_TEST = len(test_stories)

random.shuffle(train_stories)
valid_stories = train_stories[int(len(train_stories)*0.95):]
train_stories = train_stories[:int(len(train_stories)*0.95)]

print('Validation size: {0}'.format(len(valid_stories)))
print('Training size: {0}'.format(len(train_stories)))

In [ ]:
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return izip_longest(*args, fillvalue=fillvalue)

In [ ]:
acc_hist = []
acc_hist.append(0.)

show_batch_interval = 50

for k in xrange(EPOCHS):
    for b,train_batch in enumerate(zip(grouper(train_stories, BATCH_SIZE, fillvalue=train_stories[-1]))):
        X,Y = get_spacy_vectors(train_batch[0], answer_dict, 
                                         max_len, nlp)
            
        loss = model.train_on_batch(X, Y)
        if b % show_batch_interval == 0:
            print('Epoch: {0}, Batch: {1}, loss: {2}'.format(k,b,loss))
    
    X,Y = get_spacy_vectors(valid_stories, answer_dict, 
                                        max_len, nlp)
    loss, acc = model.evaluate(X, Y, batch_size=BATCH_SIZE)
    print('Epoch{0}, Valid loss / valid accuracy = {1:.4f} / {2:.4f}'.format(k,loss, acc))

    # logging results
    with open(base_dir + '/logs/log_{0}_{1}_drop_{2}.txt'.format(
        'GRU',str(NUM_HIDDEN_UNITS),str(DROPOUT_FACTOR)),'a') as fil:
        fil.write(str(loss) + ' ' + str(acc) + '\n')

    # saving model
    if max(acc_hist) < acc:
        model.save_weights(base_dir + '/models/weights_{0}_{1}_drop_{2}.hdf5'.format(
                'GRU',str(NUM_HIDDEN_UNITS),str(DROPOUT_FACTOR)),overwrite=True)

    acc_hist.append(acc)

In [ ]:
acc_5 = 0.
acc = 0.

for b,test_batch in enumerate(zip(grouper(test_stories, BATCH_SIZE, fillvalue=test_stories[-1]))):
    X,Y = get_spacy_vectors(
        test_batch[0], answer_dict, max_len, nlp
    )
    answers_test = Y if b == 0 else np.vstack((answers_test,Y))
    preds = model.predict(X)

    # Saving in order to make some more visualizations
    all_predictions = preds if b == 0 else np.vstack((all_predictions,preds))
    if b % 50 == 0:
        print('Batch: {0}'.format(b))

In [ ]:
all_predictions = all_predictions[:len(test_stories)]
answers_test = answers_test[:len(test_stories)]

for k,(pred,answer) in enumerate(zip(all_predictions, answers_test)):
    prediction = np.argsort(pred)[-5:][::-1]
    pred_words = [answer_dict.keys()[answer_dict.values().index(pred)] for pred in prediction]
    answer_word = answer_dict.keys()[answer_dict.values().index(answer.argmax())]

    if answer_word in pred_words:
        acc_5 += 1.

    if pred_words[0] == answer_word:
        acc += 1.

In [ ]:
all_err = -np.log(all_predictions[range(all_predictions.shape[0]),answers_test.argmax(axis=1)])

np.savetxt('logs/error.dat', all_err)

In [ ]:
acc /= len(test_stories)
acc_5 /= len(test_stories)

print('Accuracy: {0}'.format(acc))
print('5 most prob. accuracy: {0}'.format(acc_5))

##  VI- SEVERITY / DESCRIPTION / PRECAUTION

In [ ]:
severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

def getDescription():
    global description_list
    with open(DATASETS_DIR + 'symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open(DATASETS_DIR + 'symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open(DATASETS_DIR + 'symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


In [ ]:
getSeverityDict()
getprecautionDict()
getDescription()

In [ ]:
severityDictionary

In [ ]:
def calc_condition(exp,days):
    sum=0
    for item in exp:
        if item in severityDictionary.keys():
            sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp))>13):
        return 1
        print("You should take the consultation from doctor. ")
    else:
        return 0
        print("It might not be that bad but you should take precautions.")


# Chat

In [ ]:
def getInfo():
    # name=input("Name:")
    print("Your Name \n\t\t\t\t\t\t",end="=>")
    name=input("")
    print("hello ",name)
    return str(name)

In [ ]:
def related_sym(psym1):
    if len(psym1)==1:
        return psym1[0]
    print("searches related to input: ")
    for num,it in enumerate(psym1):
        print(num,")",clean_symp(it))
    if num!=0:
        print(f"Select the one you meant (0 - {num}):  ", end="")
        conf_inp = int(input(""))
    else:
        conf_inp=0

    disease_input=psym1[conf_inp]
    return disease_input

In [ ]:
def main_sp(name,all_symp_col):
    #main Idea: At least two initial sympts to start with
    
    #get the 1st syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)
    print("Enter the main symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym1 = input("")
    sym1=preprocess_sym(sym1)
    sim1,psym1=syntactic_similarity(sym1,all_symp_pr)
    if sim1==1:
        psym1=related_sym(psym1)
    
    #get the 2nd syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)

    print("Enter a second symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym2=input("")
    sym2=preprocess_sym(sym2)
    sim2,psym2=syntactic_similarity(sym2,all_symp_pr)
    if sim2==1:
        psym2=related_sym(psym2)
        
    #if check_pattern==0 no similar syntaxic symp1 or symp2 ->> try semantic similarity
    
    if sim1==0 or sim2==0:
        sim1,psym1=semantic_similarity(sym1,all_symp_pr)
        sim2,psym2=semantic_similarity(sym2,all_symp_pr)
        
        #if semantic sim syp1 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim1==0:
            sugg=suggest_syn(sym1)
            print('Are you experiencing any ')
            for res in sugg:
                print(res)
                inp=input('')
                if inp=="yes":
                    psym1=res
                    sim1=1
                    break
                
        #if semantic sim syp2 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim2==0:
            sugg=suggest_syn(sym2)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym2=res
                    sim2=1
                    break
        #if no syntaxic semantic and suggested sym found return None and ask for clarification

        if sim1==0 and sim2==0:
            return None,None
        else:
            # if at least one sym found ->> duplicate it and proceed
            if sim1==0:
                psym1=psym2
            if sim2==0:
                psym2=psym1
    #create patient symp list
    all_sym=[col_dict[psym1],col_dict[psym2]]
    #predict possible diseases
    diseases=possible_diseases(all_sym)
    stop=False
    print("Are you experiencing any ")
    for dis in diseases:
        print(diseases)
        if stop==False:
            for sym in symVONdisease(df_tr,dis):
                if sym not in all_sym:
                    print(clean_symp(sym)+' ?')
                    while True:
                        inp=input("")
                        if(inp=="yes" or inp=="no"):
                            break
                        else:
                            print("provide proper answers i.e. (yes/no) : ",end="")
                    if inp=="yes":
                        all_sym.append(sym)
                        diseases=possible_diseases(all_sym)
                        if len(diseases)==1:
                            stop=True 
    return knn_clf.predict(OHV(all_sym,all_symp_col)),all_sym

    
    

In [ ]:
def chat_sp():
    a=True
    while a:
        name=getInfo()
        result,sym=main_sp(name,all_symp_col)
        if result == None :
            ans3=input("can you specify more what you feel or tap q to stop the conversation")
            if ans3=="q":
                a=False
            else:
                continue

        else:
            print("you may have "+result[0])
            print(description_list[result[0]])
            an=input("how many day do you feel those symptoms ?")
            if calc_condition(sym,int(an))==1:
                print("you should take the consultation from doctor")
            else : 
                print('Take following precautions : ')
                for e in precautionDictionary[result[0]]:
                    print(e)
            print("do you need another medical consultation (yes or no)? ")
            ans=input()
            if ans!="yes":
                a=False
                print("!!!!! thanks for using ower application !!!!!! ")


In [ ]:
df_tr.iloc[-1]

In [ ]:
# import joblib
# knn_clf = joblib.load('model/knn.pkl')  

In [ ]:
symVONdisease(df_tr,"Jaundice")

In [ ]:
knn_clf.predict(OHV(['fatigue', 'weight_loss', 'itching','high_fever'],all_symp_col))

In [ ]:
d=df_tr[df_tr.iloc[:,-1]=="Fungal infection"].sum(axis=0)

In [ ]:
cl=df_tr.columns

In [ ]:
pp=d!=0

In [ ]:
cl[pp]

In [ ]:
d[pp].drop('prognosis')

In [ ]:
chat_sp()